In [1]:
#James Chartouni
from multiprocessing import Pool, cpu_count
import gc; gc.enable()
import pandas as pd
import numpy as np

In [2]:
train = pd.read_csv('cleaned_input/train_merged.csv')
test = pd.read_csv("cleaned_input/test_merged.csv")
data = pd.concat(( train, test ))
train = []
test = []

In [3]:
print("msno")
msno_values = data.msno.unique()
print("end msno")
del(data)

msno
end msno


In [4]:
counts = pd.read_csv("cleaned_input/counts.csv")

In [10]:
#get number of unique visits
user_features = pd.DataFrame() 

user_features['msno'] = np.nan #counts.index
#user_features["frequency"] = np.nan #counts.values
user_features["gap_days"] = np.nan
user_features["average_gap_days"] = np.nan
user_features["longest_gap"] = np.nan
user_features["gaps_longer_29"] = np.nan
user_features.head()

,msno,gap_days,average_gap_days,longest_gap,gaps_longer_29


In [12]:
#read in user_logs
import time
chunksize = 25000
i = 0 
for chunk in pd.read_csv('raw_data/user_logs.csv', engine='c', chunksize=chunksize, usecols=[0, 1, 7, 8], dtype={"msno":str, "date": int, "num_unq":int, "total_secs":float}):
    if i > 5000:
        start_time = time.time()
        chunk = chunk[chunk['msno'].isin(msno_values)]
        rows = chunk.msno.unique()
        chunk['date'] = pd.to_datetime(chunk['date'], format='%Y%m%d')
        #print("processing ",i,"took %s seconds" % (time.time() - start_time))
        #print(len(rows))
        for row in rows:
            row_time = time.time()
            df = chunk[chunk["msno"] == row]
            #print("row ",i,"took %s seconds" % (time.time() - row_time))
            differences = df.date.diff()
            #print("row ",i,"took %s seconds" % (time.time() - row_time))
            gap_days = differences.sum()
            #print("row ",i,"took %s seconds" % (time.time() - row_time))
            average_gap_days = differences.mean()
            #print("row ",i,"took %s seconds" % (time.time() - row_time))
            if len(differences.nlargest(1)) > 0:
                longest_gap = differences.nlargest(1).iloc[0].total_seconds()
            else:
                longest_gap = -1
            #print("row ",i,"took %s seconds" % (time.time() - row_time))
            gaps_longer_29 = 0
            for val in differences:
                if val > pd.Timedelta('29 days'):
                    gaps_longer_29 += 1
            #print("row ",i,"took %s seconds" % (time.time() - row_time))
            user_features.loc[len(user_features)] = [row, gap_days, average_gap_days.total_seconds(), longest_gap, gaps_longer_29]
            #print("row ",i,"took %s seconds" % (time.time() - row_time))
        #del(chunk)
        print("Round ",i,"took %s seconds" % (time.time() - start_time))
        if i % 100 == 0:
            user_features.to_csv(str(i) + "_user_logs_features_3.csv")
    i += 1 
user_features.head()

Round  5001 took 14.779730796813965 seconds
Round  5002 took 11.931550979614258 seconds
Round  5003 took 16.79717206954956 seconds
Round  5004 took 14.18759799003601 seconds
Round  5005 took 15.086220026016235 seconds
Round  5006 took 18.86276912689209 seconds
Round  5007 took 13.98813796043396 seconds
Round  5008 took 17.83205485343933 seconds
Round  5009 took 18.541889905929565 seconds
Round  5010 took 14.437794923782349 seconds
Round  5011 took 21.024388074874878 seconds
Round  5012 took 20.24731683731079 seconds
Round  5013 took 15.991447687149048 seconds
Round  5014 took 21.657865285873413 seconds
Round  5015 took 19.925588846206665 seconds
Round  5016 took 17.50423789024353 seconds
Round  5017 took 22.17165207862854 seconds
Round  5018 took 18.050588846206665 seconds
Round  5019 took 20.091863870620728 seconds
Round  5020 took 24.161839246749878 seconds
Round  5021 took 18.021989822387695 seconds
Round  5022 took 27.26093578338623 seconds
Round  5023 took 25.088167190551758 secon

KeyboardInterrupt: 